# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
features = ['Was_home',
 'Team_rating',
 'Opp_rating',
 'Rating_difference',
 'Min_2',
 'Start_2',
 'Gls_2',
 'Sh_2',
 'SoT_2',
 'Ast_2',
 'CrdY_2',
 'CrdR_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'ICT_index_2',
 'FPL_points_2',
 'Baseline_points_2',
 'Bonus_2',
 'Min_4',
 'Start_4',
 'Gls_4',
 'Sh_4',
 'SoT_4',
 'Ast_4',
 'CrdY_4',
 'CrdR_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'ICT_index_4',
 'FPL_points_4',
 'Baseline_points_4',
 'Bonus_4',
 'Min_30',
 'Start_30',
 'Gls_30',
 'Sh_30',
 'SoT_30',
 'Ast_30',
 'CrdY_30',
 'CrdR_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'ICT_index_30',
 'FPL_points_30',
 'Baseline_points_30',
 'Bonus_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(features)

61

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
X_test = df[df["Season"] == CURRENT_SEASON][features]
# .reset_index(drop=True)

In [7]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Baseline_points_2',
       'Baseline_points_30', 'Baseline_points_4', 'Bonus_2', 'Bonus_30',
       'Bonus_4', 'CrdR_2', 'CrdR_30', 'CrdR_4', 'CrdY_2', 'CrdY_30', 'CrdY_4',
       'DEF', 'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GW', 'Gls_2', 'Gls_30', 'Gls_4', 'ICT_index_2',
       'ICT_index_30', 'ICT_index_4', 'MID', 'Min_2', 'Min_30', 'Min_4',
       'Name_original', 'Opp_rating', 'Opp_score_2', 'Opp_score_30',
       'Opp_score_4', 'Opponent', 'Rating_difference', 'Season', 'Sh_2',
       'Sh_30', 'Sh_4', 'SoT_2', 'SoT_30', 'SoT_4', 'Start_2', 'Start_30',
       'Start_4', 'Team', 'Team_CS_2', 'Team_CS_30', 'Team_CS_4',
       'Team_rating', 'Team_result_2', 'Team_result_30', 'Team_result_4',
       'Team_score_2', 'Team_score_30', 'Team_score_4', 'Was_home', 'xA_2',
       'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'Finished', 'Date', 'Price'],
      dtype='object')

In [8]:
X_test.columns

Index(['Was_home', 'Team_rating', 'Opp_rating', 'Rating_difference', 'Min_2',
       'Start_2', 'Gls_2', 'Sh_2', 'SoT_2', 'Ast_2', 'CrdY_2', 'CrdR_2',
       'xG_2', 'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2',
       'Team_result_2', 'ICT_index_2', 'FPL_points_2', 'Baseline_points_2',
       'Bonus_2', 'Min_4', 'Start_4', 'Gls_4', 'Sh_4', 'SoT_4', 'Ast_4',
       'CrdY_4', 'CrdR_4', 'xG_4', 'xA_4', 'Team_CS_4', 'Team_score_4',
       'Opp_score_4', 'Team_result_4', 'ICT_index_4', 'FPL_points_4',
       'Baseline_points_4', 'Bonus_4', 'Min_30', 'Start_30', 'Gls_30', 'Sh_30',
       'SoT_30', 'Ast_30', 'CrdY_30', 'CrdR_30', 'xG_30', 'xA_30',
       'Team_CS_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30',
       'ICT_index_30', 'FPL_points_30', 'Baseline_points_30', 'Bonus_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [9]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [10]:
model

GradientBoostingRegressor(random_state=42)

# Predicting

In [11]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [12]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [13]:
preds = get_predictions(model, df, X_test)

14507 (14507, 72)


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [15]:
# next gameweek
preds[preds["GW"] == 2].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
4401,Erling Haaland,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,FWD,140.0,10.551900
9284,Marcus Rashford,2.0,2023-24,Manchester Utd,Tottenham,0.0,1877.037842,1817.656738,MID,90.0,7.867932
7660,Julián Álvarez,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,FWD,65.0,7.694637
4892,Gabriel Martinelli Silva,2.0,2023-24,Arsenal,Crystal Palace,0.0,1921.462769,1763.994995,MID,80.0,7.550077
7131,John McGinn,2.0,2023-24,Aston Villa,Everton,1.0,1816.929932,1699.505859,MID,55.0,7.420167
10193,Mohamed Salah,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,MID,125.0,7.363090
532,Alexis Mac Allister,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,MID,60.0,7.094179
11405,Pervis Estupiñán,2.0,2023-24,Brighton,Wolves,0.0,1832.364258,1715.631348,DEF,51.0,7.050610
11291,Pascal Groß,2.0,2023-24,Brighton,Wolves,0.0,1832.364258,1715.631348,MID,65.0,6.973517
7772,Kaoru Mitoma,2.0,2023-24,Brighton,Wolves,0.0,1832.364258,1715.631348,MID,65.0,6.889995


In [16]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
4431,Erling Haaland,33.0,2023-24,Manchester City,Luton,1.0,2080.850586,1602.327637,FWD,140.0,11.566655
4428,Erling Haaland,30.0,2023-24,Manchester City,Arsenal,1.0,2080.850586,1921.462769,FWD,140.0,10.599805
4401,Erling Haaland,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,FWD,140.0,10.551900
4410,Erling Haaland,11.0,2023-24,Manchester City,Bournemouth,1.0,2080.850586,1662.536621,FWD,140.0,10.548409
4420,Erling Haaland,22.0,2023-24,Manchester City,Burnley,1.0,2080.850586,1722.802979,FWD,140.0,10.533860
4422,Erling Haaland,24.0,2023-24,Manchester City,Everton,1.0,2080.850586,1699.505859,FWD,140.0,10.533860
4416,Erling Haaland,17.0,2023-24,Manchester City,Crystal Palace,1.0,2080.850586,1763.994995,FWD,140.0,10.533860
4434,Erling Haaland,36.0,2023-24,Manchester City,Wolves,1.0,2080.850586,1715.631348,FWD,140.0,10.533860
4403,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2080.850586,1745.365723,FWD,140.0,10.533860
4408,Erling Haaland,9.0,2023-24,Manchester City,Brighton,1.0,2080.850586,1832.364258,FWD,140.0,10.486730


In [17]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Erling Haaland,364.038344
Gabriel Martinelli Silva,285.615995
Marcus Rashford,284.870557
Sean Longstaff,271.176815
Martin Ødegaard,268.610379
Mohamed Salah,262.353685
Kai Havertz,261.172336
Julián Álvarez,260.914288
Pervis Estupiñán,255.585704


# Saving to file

In [18]:
preds.to_csv("predictions/preds_next_season.csv", index=False)